In [ ]:
import requests
import json
from datetime import datetime, timedelta, timezone
import pandas as pd
import os
import re

In [ ]:
import warnings

warnings.filterwarnings("ignore")

## Functions


In [ ]:
# Merge files
def merge_csv_files(directory_path):

    """
    Merges all CSV files in a directory into a single pandas DataFrame.

    Args:
    directory_path (str): The directory path containing the CSV files.

    Returns:
    merged_df (pandas.DataFrame): The merged pandas DataFrame of all CSV files in the directory.
    """
    dfs = []

    # loop through each file in the directory
    for file in os.listdir(directory_path):
        # check if the file is a CSV file
        if file.endswith(".csv"):
            file_path = os.path.join(directory_path, file)
            if directory_path.endswith("dataverse_file"):
                df = pd.read_csv(file_path, delimiter=",")
            else:
                df = pd.read_csv(file_path, delimiter=";")
            dfs.append(df)

    # concatenate all dataframes
    merged_df = pd.concat(dfs, ignore_index=True)

    return merged_df

## Get historical air quality data

In [ ]:
# Get air quality data for Leuven
# Period: Jan 1, 2022 to Dec 31, 2022
api_key = "ec722f11f234fb9a316e2580e6e2019e"
lat = 50.88
lon = 4.7
start_date = "1640995200"
end_date = "1672531200"

url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_date}&end={end_date}&appid={api_key}"
# url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={api_key}"
# url = f"http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={api_key}"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print("Error:", response.status_code)

In [ ]:
# Historical
df = pd.DataFrame(data["list"])
df["dt"] = pd.to_datetime(df["dt"], unit="s")
components_df = pd.json_normalize(df["components"])
aqi_df = pd.json_normalize(df["main"])
df = pd.concat([df, components_df, aqi_df], axis=1)
df.drop(["components", "main"], axis=1, inplace=True)
df.head(10)

## Process air quality data

In [ ]:
air_quality_data = df

# Format time stamp
air_quality_data["dt"] = pd.to_datetime(air_quality_data["dt"])
air_quality_data["date"] = air_quality_data["dt"].dt.date
air_quality_data["hour"] = air_quality_data["dt"].dt.hour
air_quality_data["month"] = air_quality_data["dt"].dt.month
air_quality_data["weekday"] = air_quality_data["dt"].dt.strftime("%a")

air_quality_data.to_csv("../data/processed_air_quality_data.csv")

## Get historical traffic data

In [ ]:
url = "https://telraam-api.net/v1/reports/traffic"
headers = {"X-Api-Key": "Z9Qoyuy9Yf99nf2I0Myig4t0ftuUmY81ahBIVZH4"}

start_date = pd.to_datetime("2022-01-01")
end_date = pd.to_datetime("2023-01-01")

# define the duration for each loop (3 months)
loop_duration = pd.DateOffset(months=3)

# define the list of sensor IDs
counter_ids = [
    "9000000627",
    "347690",
    "9000000674",
    "9000000773",
    "347931",
    "347860",
    "9000000764",
    "9000000672",
    "347948",
    "9000001547",
    "347365",
    "349054",
    "9000000681",
]

full_data = pd.DataFrame()

# loop through the counter IDs
for counter_id in counter_ids:
    # retrieve data for each 3-month period
    current_date = start_date
    while current_date < end_date:
        loop_start_date = current_date
        loop_end_date = loop_start_date + loop_duration

        body = {
            "id": counter_id,
            "time_start": loop_start_date.strftime("%Y-%m-%d %H:%M:%SZ"),
            "time_end": loop_end_date.strftime("%Y-%m-%d %H:%M:%SZ"),
            "level": "segments",
            "format": "per-hour",
        }
        payload = str(body)

        response = requests.post(url, headers=headers, data=payload)
        json_data = response.json()
        loop_data = pd.DataFrame(json_data["report"])
        full_data = pd.concat([full_data, loop_data])

        # update the current date for the next loop
        current_date += loop_duration

In [ ]:
location = {
    "9000000627": "redingenhof",
    "347690": "Kapucijnenvoer",
    "9000000674": "Tiensestraat",
    "9000000773": "Bondgenotenlaan",
    "347931": "Vital Decostersstraat",
    "347860": "BROUWERSSTRAAT",
    "9000000764": "Fonteinstraat 137 b 301",
    "9000000672": "Petermannenstraat",
    "347948": "Ridderstraat",
    "9000001547": "Jan Pieter Minckelersstraat",
    "347365": "Dekenstraat",
    "349054": "Pleinstraat",
    "9000000681": "Bierbeekstraat",
}

In [ ]:
full_data["location"] = full_data.segment_id.astype("str").map(location)
full_data.to_csv("../data/traffic_leuven_2022.csv", index=False)

In [ ]:
full_data

## Get historical weather data

In [ ]:
def get_forecast_hourly_weather(url):
    resp = requests.get(url)
    data = resp.json()
    df = pd.DataFrame(data["hourly"])
    return df

In [ ]:
url = "https://archive-api.open-meteo.com/v1/archive?latitude=50.88&longitude=4.70&start_date=2022-01-01&end_date=2022-12-31&timezone=Europe%2FBerlin&hourly=temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,pressure_msl,surface_pressure,precipitation,snowfall,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,windspeed_10m,winddirection_10m,windgusts_10m"
weather_data = get_forecast_hourly_weather(url)

## Process historical weather data

In [ ]:
## rename the column
weather_data.columns = [re.sub(" \(.*\)", "", col) for col in weather_data.columns]

# Format time stamp
weather_data["time"] = pd.to_datetime(weather_data["time"])
weather_data["date"] = weather_data["time"].dt.date
weather_data["hour"] = weather_data["time"].dt.hour
weather_data["month"] = weather_data["time"].dt.month
weather_data["weekday"] = weather_data["time"].dt.strftime("%a")

weather_data.to_csv("../data/processed_weather_data_leuven.csv")

In [ ]:
weather_data["temperature_2m"].hist()

## Process file 40 

In [ ]:
file40 = merge_csv_files("../data/file40")

In [ ]:
### we would delete unit column because of same values
# drop all _unit columns
cols_to_drop = [col for col in file40.columns if col.endswith("unit")]
file40.drop(cols_to_drop, axis=1, inplace=True)

# rename columns
file40.rename(
    columns={"description": "location", "#object_id": "object_id"}, inplace=True
)

# Convert the 'result_timestamp' column to a datetime data type
file40["result_timestamp"] = pd.to_datetime(file40["result_timestamp"])
file40["date"] = file40["result_timestamp"].dt.date
file40["hour"] = file40["result_timestamp"].dt.hour
file40["month"] = file40["result_timestamp"].dt.month
file40["weekday"] = file40["result_timestamp"].dt.strftime("%a")

file40.to_csv("../data/processed_file40_data.csv")

## Process file 41

In [ ]:
# Merge files
file41 = merge_csv_files("../data/file41")

In [ ]:
# drop unncessary cols
cols_to_drop = [
    "noise_event_laeq_model_id_unit",
    "noise_event_laeq_model_id",
    "noise_event_laeq_primary_detected_certainty_unit",
    "noise_event_laeq_primary_detected_class_unit",
]

file41.drop(cols_to_drop, axis=1, inplace=True)

# rename cols
file41.columns = [
    "object_id",
    "location",
    "result_timestamp",
    "noise_event_certainty",
    "noise_event",
]

# remove the noise_event that are unsupported
file41 = file41.loc[file41.noise_event != "Unsupported"]

In [ ]:
# extract from timestamp
file41["result_timestamp"] = pd.to_datetime(file41["result_timestamp"])
file41["time"] = file41["result_timestamp"].dt.time
file41["date"] = file41["result_timestamp"].dt.date
file41["hour"] = file41["result_timestamp"].dt.hour
file41["month"] = file41["result_timestamp"].dt.month
file41["weekday"] = file41["result_timestamp"].dt.strftime("%a")

In [ ]:
# Only adding datapoints with certanity of 85%+
file41 = pd.DataFrame(file41.loc[file41["noise_event_certainty"] > 85].reset_index())
# Dropping undefined columns
file41 = file41.dropna(subset=["noise_event"])
# Pivot table to transform into counting hourly types of noise events
file41_piv = (
    pd.pivot_table(
        file41,
        index=["object_id", "date", "hour", "month", "weekday"],
        columns=["noise_event"],
        aggfunc="count",
    )
    .xs("location", level=0, axis=1)
    .reset_index()
)
file41_piv.fillna(0, inplace=True)

file41_piv.to_csv("../data/processed_file41_data")

## Process file 42 - full data

In [ ]:
#folder containing the full data
main_folder = "s3://teamchadmda/export_42_full/"

In [ ]:
#list of months and their numerical codes used in the data structure
months = [
    'Jan', 'Feb', 'March', 'April', 'May', 'June',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
]
num_codes = range(42,53)
num_codes = [str(num) for num in num_codes]
month_codes = {month: code for month,code in zip(months,num_codes)}
month_codes['Feb'] = '42'

#dictionary of location codes
object_id_dict = {
    255439: "MP 01: Naamsestraat 35 Maxim",
    255440: "MP 02: Naamsestraat 57 Xior",
    255441: "MP 03: Naamsestraat 62 Taste",
    303910: "MP 04: His & Hears",
    255442: "MP 05: Calvariekapel KU Leuven",
    255443: "MP 06: Parkstraat 2 La Filosovia",
    255444: "MP 07: Naamsestraat 81",
    280324: "MP08bis - Vrijthof",
}

#File name list, the stuff that comes after csv_results_[monthcode]
file_name_exts = ["_255439_mp-01-naamsestraat-35-maxim.csv",
                 "_255440_mp-02-naamsestraat-57-xior.csv",
                 "_255441_mp-03-naamsestraat-62-taste.csv",
                 "_255442_mp-05-calvariekapel-ku-leuven.csv",
                 "_255443_mp-06-parkstraat-2-la-filosovia.csv",
                 "_255444_mp-07-naamsestraat-81.csv",
                 "_255445_mp-08-kiosk-stadspark.csv",
                 "_280324_mp08bis---vrijthof.csv",
                 "_303910_mp-04-his-hears.csv",]

#function to give the list of files for a month's folder
def filelist(monthstring):
    filelist = []
    for file in file_name_exts:
        filelist.append(main_folder+monthstring+"/csv_results_"+month_codes[monthstring]+file)
    return filelist

In [ ]:
filelist('March')

In [ ]:
#function to process a given file in the file42 directory

def process_file(file_path):
    print(f"Processing: {file_path}")
    df = pd.read_csv(file_path, delimiter=";")
    
    # convert 'result_timestamp' to datetime format
    df["result_timestamp"] = pd.to_datetime(
        df["result_timestamp"], format="%d/%m/%Y %H:%M:%S.%f"
    )
    
    # set 'result_timestamp' as the index
    df.set_index("result_timestamp", inplace=True)
    
    # resample to hourly frequency and calculate the mean
    df_resampled = df.resample("H").mean()
    
    # reset index and add additional columns
    df_resampled.reset_index(inplace=True)
    df_resampled["date"] = df_resampled["result_timestamp"].dt.date
    df_resampled["hour"] = df_resampled["result_timestamp"].dt.hour
    df_resampled["weekday"] = df_resampled["result_timestamp"].dt.strftime("%a")
    df_resampled["month"] = df_resampled["result_timestamp"].dt.month
    
    # drop rows with NaN values in 'lamax' column
    df_resampled.dropna(subset=["lamax"], inplace=True)
    
    # rename column and convert 'object_id' to int
    df_resampled.rename(columns={"#object_id": "object_id"}, inplace=True)
    df_resampled["object_id"] = df_resampled["object_id"].astype(int)
    
    # ddd location column using 'object_id_dict'
    df_resampled["location"] = df_resampled["object_id"].map(object_id_dict)
    return df_resampled

In [ ]:
#function that goes through entire filesystem, processes files, and gathers it all together

def gather_file_42():
    concatenated_df = pd.DataFrame()
    
    for month in months:
        file_list = filelist(month)
        
        for file in file_list:
            df_resampled = process_file(file)
            # concatenate
            concatenated_df = pd.concat([concatenated_df, df_resampled])
            concatenated_df = concatenated_df[
                [
                    "result_timestamp",
                    "object_id",
                    "lamax",
                    "laeq",
                    "lceq",
                    "lcpeak",
                    "date",
                    "hour",
                    "weekday",
                    "month",
                    "location",
                ]
            ]
    return concatenated_df

In [ ]:
file42 = gather_file_42()
file42.to_csv("../data/processed_file42.csv")